In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv('DSA_features.csv')

In [3]:
df

,T_xacc_mean,T_xacc_max,T_xacc_min,T_xacc_var,T_xacc_std,T_xacc_skew,T_yacc_mean,T_yacc_max,T_yacc_min,T_yacc_var,...,LL_ymag_std,LL_ymag_skew,LL_zmag_mean,LL_zmag_max,LL_zmag_min,LL_zmag_var,LL_zmag_std,LL_zmag_skew,activity,people
0,7.975714,8.1605,7.6823,0.014395,0.119981,-0.023319,1.083150,1.1832,0.99744,0.002208,...,0.000792,0.177075,-0.057119,-0.054963,-0.059241,6.778722e-07,0.000823,0.036729,sitting,p1
1,7.978250,8.1763,7.8472,0.007551,0.086896,0.552416,1.140865,1.2129,1.05810,0.000784,...,0.000860,-0.286918,-0.057268,-0.054945,-0.059589,7.032302e-07,0.000839,0.347471,sitting,p1
2,7.970894,8.0860,7.8470,0.003092,0.055603,0.100538,1.140962,1.2128,1.07960,0.000508,...,0.000762,-0.134430,-0.057068,-0.054711,-0.059065,6.268222e-07,0.000792,0.045579,sitting,p1
3,7.938412,8.1083,7.6901,0.003763,0.061343,-0.231914,1.165260,1.3170,1.07870,0.002173,...,0.000735,0.021485,-0.056422,-0.053670,-0.058310,8.011245e-07,0.000895,0.240690,sitting,p1
4,7.908930,8.1305,7.8322,0.001741,0.041731,2.042285,1.187504,1.2574,1.09450,0.000662,...,0.000824,-0.148229,-0.055801,-0.053313,-0.057815,6.853423e-07,0.000828,0.258429,sitting,p1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9115,8.280854,34.1980,-2.9038,28.080803,5.299132,1.350075,-1.491537,11.2240,-11.65100,14.670334,...,0.200829,-0.040701,0.297666,0.708480,-0.117430,4.135451e-02,0.203358,-0.310022,basketBall,p8
9116,9.591118,51.6970,-3.4129,35.722025,5.976791,2.981144,0.086304,6.9951,-11.76400,5.329897,...,0.148745,-0.266377,0.224716,0.554670,-0.250950,3.355704e-02,0.183186,-0.736410,basketBall,p8
9117,9.599113,27.9300,-1.0765,48.850886,6.989341,0.449237,-0.728367,3.7801,-8.36910,5.683022,...,0.310748,-0.009505,-0.237786,0.088854,-0.477260,2.026107e-02,0.142341,0.668438,basketBall,p8
9118,9.692482,72.7820,-2.6734,59.378336,7.705734,4.491114,-0.582724,6.1216,-8.85710,4.162963,...,0.156493,0.050624,0.533023,0.677800,0.055941,1.356379e-02,0.116464,-1.482489,basketBall,p8


In [4]:
df.isnull().values.any()

False

In [5]:
normal_data = df[df['activity'].isin(['lyingRight', 'lyingBack'])]
abnormal_data = df[df['activity'] == 'sitting']

In [6]:
test_data = pd.concat([normal_data, abnormal_data])

In [7]:
X = test_data.drop(columns=['activity', 'people']).values

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
input_dim = X_scaled.shape[1]

In [11]:
input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded) 

In [13]:
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

In [14]:
autoencoder = Model(input_layer, decoded)

In [15]:
encoder = Model(input_layer, encoded)

In [16]:
encoded_input = Input(shape=(32,))
decoder_layer = autoencoder.layers[-2](encoded_input)
decoder_layer = autoencoder.layers[-1](decoder_layer)
decoder = Model(encoded_input, decoder_layer)


In [17]:
autoencoder.compile(optimizer='adam', loss='mse')

In [18]:
autoencoder.fit(X_scaled, X_scaled, epochs=50, batch_size=256, validation_split=0.2)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 1.1636 - val_loss: 1.4449
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1853 - val_loss: 1.4343
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.1098 - val_loss: 1.4217
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.2096 - val_loss: 1.4059
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.2523 - val_loss: 1.3859
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1385 - val_loss: 1.3608
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.2629 - val_loss: 1.3313
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.1218 - val_loss: 1.2995
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.0521 - val_loss: 1.2684
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.9932 - val_loss: 1.2415
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.8970 - val_loss: 1.2207
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.8946 - val_loss: 1.2055
Epoch 13/50

In [30]:
reconstructed_data = autoencoder.predict(X_scaled)

mse = np.mean(np.power(X_scaled - reconstructed_data, 2), axis=1)
print(f"Mean Squared Error: {mse}")

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step
Mean Squared Error: [ 0.21296465  0.23485055  0.19467794  0.24202625  0.21856105  0.21063183
  0.1967948   0.18553359  0.19074995  0.24326786  0.19883831  0.19080265
  0.22266172  0.17701667  0.21948678  0.18862262  0.17577617  0.18750747
  0.19819511  0.18872686  0.22266931  0.17470041  0.220745    0.44573542
  0.19645928  0.19204282  0.19511306  0.39221015  0.55290562  0.50206062
  0.44354775  0.19234703  0.31591092  0.30147436  0.2797406   0.1732733
  0.20032939  0.25586756  0.23187211  0.17703703  0.18744857  0.19454488
  0.33538601  0.2040091   0.30441911  0.23114944  0.48263769  0.31872465
  0.42475927  0.25413693  0.22885742  0.17038965  0.21523779  0.19911895
  0.1864275   0.19332532  0.20538504  0.17179657  0.16918235  0.17055454
  1.90415368 70.62395304  2.70921251  3.51821078  1.65962697  1.08077483
  0.77615113  0.75754762  0.76541625  0.73764503  1.42810095  0.82214576
  0.79618545  0.74661482  0.73486894  1.37100916  2.57072268  0.

In [31]:
threshold = 0.3
anomalies = mse > threshold
print(f"탐지된 이상치 개수: {np.sum(anomalies)}")

탐지된 이상치 개수: 457
